### This script will open up the results of the hyperparameter search output.  Used to make adjustments and re-run the entire process quicker.

In [ ]:
# imports
import numpy as np
import pandas as pd
import glob

In [2]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import numba

import time
import datetime

import platform
import logging
import sys
import os
from os import path
# works across all three OS
from sys import platform

import math
import string
import gc

import warnings
import itertools

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf


import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# working directory / files
workDir = 'D:\\project\\data\\kg_corpgroc\\'
eportDir = 'D:\\project\\data\\kg_corpgroc\\export\\completed\\'

In [4]:
# Load test / train
directory = 'D:\\project\\data\\kg_corpgroc\\'
exportDirectory = directory + 'export\\'

file_test = 'test.csv'

#file_train = 'train.csv'
CONST_USE_SMALLER_MEMORY_FILE = True
if CONST_USE_SMALLER_MEMORY_FILE == True:
    file_train = 'train_20160801_20170814.csv'
else:
    file_train = 'train_20160801_20170901_imputed.csv'

# set the column data types for memory efficiency
coltypes_train = {'id': 'int64',
                  # 't_date':'datetime64',
                  'store_nbr': 'int8',
                  'item_nbr': 'int32',
                  'unit_sales': 'float32',
                  'promotion': 'bool'
                  }

coltypes_test = {'id': 'int64',
                 # 't_date':'datetime64',
                 'store_nbr': 'int8',
                 'item_nbr': 'int32',
                 # 'unit_sales':'float64'
                 'promotion': 'bool'
                 }


df_train = pd.read_csv(directory + file_train, dtype=coltypes_train, parse_dates=['date'],infer_datetime_format=True)
df_test = pd.read_csv(directory + file_test, dtype=coltypes_test, parse_dates=['date'], infer_datetime_format=True)

df_train.set_index('date', inplace=True)
df_test.set_index('date', inplace=True)

df_train.sort_values(['store_nbr', 'item_nbr'], ascending=[True, True], inplace=True)

if CONST_USE_SMALLER_MEMORY_FILE==True:
    idx = pd.DataFrame(pd.date_range('2016-08-01', '2017-09-01'), columns={'dateRange'})
    idx.set_index('dateRange', inplace=True)

In [5]:
# Becuase I split the work up on multiple machines - broke the data out by individual store.
#  Here I am going to merge all the grocery store results/files back together
    
# get data file names
filenames = glob.glob(eportDir + '/*param*')    
    
dfs = []
for filename in filenames:
    print(filename)
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)

D:\project\data\kg_corpgroc\export\completed\export_param_1.csv
D:\project\data\kg_corpgroc\export\completed\export_param_10.csv
D:\project\data\kg_corpgroc\export\completed\export_param_11.csv
D:\project\data\kg_corpgroc\export\completed\export_param_12.csv
D:\project\data\kg_corpgroc\export\completed\export_param_13.csv
D:\project\data\kg_corpgroc\export\completed\export_param_14.csv
D:\project\data\kg_corpgroc\export\completed\export_param_15.csv
D:\project\data\kg_corpgroc\export\completed\export_param_16.csv
D:\project\data\kg_corpgroc\export\completed\export_param_17.csv
D:\project\data\kg_corpgroc\export\completed\export_param_18.csv
D:\project\data\kg_corpgroc\export\completed\export_param_19.csv
D:\project\data\kg_corpgroc\export\completed\export_param_2.csv
D:\project\data\kg_corpgroc\export\completed\export_param_20.csv
D:\project\data\kg_corpgroc\export\completed\export_param_21.csv
D:\project\data\kg_corpgroc\export\completed\export_param_22.csv
D:\project\data\kg_corpgroc

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,1,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


D:\project\data\kg_corpgroc\export\completed\export_param_27.csv
D:\project\data\kg_corpgroc\export\completed\export_param_28.csv
D:\project\data\kg_corpgroc\export\completed\export_param_29.csv
D:\project\data\kg_corpgroc\export\completed\export_param_3.csv
D:\project\data\kg_corpgroc\export\completed\export_param_30.csv
D:\project\data\kg_corpgroc\export\completed\export_param_31.csv
D:\project\data\kg_corpgroc\export\completed\export_param_32.csv
D:\project\data\kg_corpgroc\export\completed\export_param_33.csv
D:\project\data\kg_corpgroc\export\completed\export_param_34.csv
D:\project\data\kg_corpgroc\export\completed\export_param_35.csv
D:\project\data\kg_corpgroc\export\completed\export_param_36.csv
D:\project\data\kg_corpgroc\export\completed\export_param_37.csv
D:\project\data\kg_corpgroc\export\completed\export_param_38.csv
D:\project\data\kg_corpgroc\export\completed\export_param_39.csv
D:\project\data\kg_corpgroc\export\completed\export_param_4.csv
D:\project\data\kg_corpgroc

In [7]:
# Order the results by the least mean squared error
big_frame.sort_values(['store_nbr','item_nbr','mse_frcst'], ascending=[True, True, True], inplace=True)

In [9]:
# Create a new data frame to start manipulating until I got the group correct
dfGroup = big_frame.copy()

In [10]:
dfGroup.set_index(['store_nbr','item_nbr'], inplace=True)

In [11]:
dfGroup.head(3)

data_present model_order model_seasonal_order model_aic  \
store_nbr item_nbr                                                           
1         96995             YES   (1, 0, 1)         (1, 1, 1, 3)   340.714   
          96995             YES   (0, 0, 1)         (1, 1, 1, 3)   339.099   
          96995             YES   (1, 1, 1)         (0, 0, 1, 3)   334.244   

                   mse_frcst mse_frcst_rnd sse_frcst sse_frcst_rnd  
store_nbr item_nbr                                                  
1         96995     0.370515      0.921053   28.1591            70  
          96995     0.371565      0.907895   28.2389            69  
          96995      0.37194      0.960526   28.2675            73

In [12]:
# Group a new dataframe based on the store and item number.  I am pulling the top 1 becuase it is ordered by the MSSE
dfTopMSE = dfGroup.groupby(['store_nbr','item_nbr']).head(1).copy()

In [13]:
# Quick visual check
dfTopMSE.head(10)

data_present model_order model_seasonal_order model_aic  \
store_nbr item_nbr                                                           
1         96995             YES   (1, 0, 1)         (1, 1, 1, 3)   340.714   
          99197             YES   (1, 0, 1)         (1, 0, 1, 7)   1089.27   
          103520            YES   (1, 0, 1)         (1, 0, 0, 7)   2428.07   
          103665            YES   (1, 0, 1)         (1, 1, 1, 7)   1744.28   
          105574            YES   (0, 1, 1)        (0, 1, 1, 14)   1895.82   
          105575            YES   (0, 0, 0)         (0, 1, 1, 7)   2402.49   
          105577            YES   (0, 1, 1)         (0, 1, 1, 7)   1343.04   
          105693            YES   (1, 0, 1)         (1, 1, 1, 3)    858.35   
          105737            YES   (0, 0, 0)        (1, 0, 1, 14)   1993.45   
          105857            YES   (1, 1, 1)         (1, 0, 1, 7)   2004.57   

                   mse_frcst mse_frcst_rnd sse_frcst sse_frcst_rnd  
store_nbr item_nbr                                                  
1         96995     0.370515      0.921053   28.1591            70  
          99197        3.537       3.82895   268.812           291  
          103520     2.00149          2.25   152.113           171  
          103665     5.38462       6.05263   409.231           460  
          105574     7.30222       7.56579   554.969           575  
          105575     22.6703       22.9737   1722.94          1746  
          105577     1.88962       2.36842   143.611           180  
          105693    0.401658      0.763158    30.526            58  
          105737     4.96864       4.84211   377.616           368  
          105857     11.0478       11.6711   839.632           887

In [14]:
# Function to help concat the arguments
def fn_concat_args(input_store_nbr, input_item_nbr, input_ts_order, input_ts_seasonal_order, input_start_predict):
    '''
        temp_store_nbr = 1
        temp_item_nbr = 96995
        temp_ts_order=(1,1,0)
        temp_ts_seasonal_order=(0,0,0,14)
        temp_start_predict = 300
    '''
    args = [[input_store_nbr, input_item_nbr, input_ts_order, input_ts_seasonal_order, input_start_predict]]
    return args

# helper to help print the time
def print_elapsed_time(statement, elapsed_time, optSuppressPrint = 0):
    if elapsed_time > 60:
        timeElapsed = statement + ' : ' + str(int(round(elapsed_time / 60))) + ' min and ' + str(int(round(elapsed_time / (60 * 2)) )) + ' seconds'

        if optSuppressPrint == 0:
            print(timeElapsed)
        return (timeElapsed)
    else:
        timeElapsed = statement + ' : ' + str(int(round(elapsed_time))) + ' seconds'
        if optSuppressPrint == 0:
            print(timeElapsed)
        return (timeElapsed)

In [16]:
# Function to run the time-series
def fn_config_timeseries(args):
    
    warnings.filterwarnings("ignore")

    # Need to split up the args
    # ensure prefixed with arg_ to avoid other test items
    arg_storeNumber = args[0]
    arg_itemNumber = args[1]
    arg_ts_order = args[2]
    arg_ts_seasonal_order = args[3]
    arg_start_predict = args[4]


    # filter the TS criteria
    dfTS = df_train[(df_train['store_nbr'] == arg_storeNumber) & (df_train['item_nbr'] == arg_itemNumber)].copy()

    # If there is no data to plot then exit
    if len(dfTS) > 0:

        # If we are using a smaller memory cycle
        if CONST_USE_SMALLER_MEMORY_FILE == True:

            # Imput the missing dates going all the way back
            dfTS = pd.concat([dfTS, idx], axis=1)
            dfTS = dfTS.asfreq('D')

            # When we joined to dates, now replace everything else for the single unit we are working on
            dfTS['id'].replace({np.nan: 0}, inplace=True)
            dfTS['item_nbr'].replace({np.nan: arg_itemNumber}, inplace=True)
            dfTS['store_nbr'].replace({np.nan: arg_storeNumber}, inplace=True)
            dfTS['unit_sales'].replace({np.nan: 0}, inplace=True)
            dfTS['onpromotion'].replace({np.nan: False}, inplace=True)


        # the forecasting
        model = sm.tsa.statespace.SARIMAX(dfTS['unit_sales'], order=arg_ts_order, seasonal_order=arg_ts_seasonal_order,
                                          enforce_stationarity = False, enforce_invertibility = False)
        # Just a test of the function
        # model = sm.tsa.statespace.SARIMAX(dfTS['unit_sales'], order=(1,2,0), seasonal_order=(1,1,1,7))

        #results = model.fit()
        results = model.fit(disp=0)

        # for 2013-01-01 to current.  Use 1600 / 2150
        # for 2016-08-01 to current.  Use 0 / 396
        # for 2016-08-01 to 2017-08-15 Use 380
        # for 2017-06-01 to 2017-08-30 - start 300 / end - 396
        # for 2017-06-01 to 2017-08-15 - start 300 / end - 381
        dfTS['forecast'] = results.predict(start=arg_start_predict, end=396)  # , dynamic=True
        
        # Round the forecast (most of the items are in whole units) - submission didn't show any difference
        dfTS['forecast_rnd'] = dfTS['forecast']
        dfTS['forecast_rnd'].replace({np.nan: 0}, inplace=True)
        dfTS['forecast_rnd'] = dfTS['forecast_rnd'].apply(math.ceil)
        
        # plot - optional for a global variable
        if optionalPlotFunctionTS == 1:
            dfTS[['unit_sales', 'forecast_rnd']]["2017-06-01":].plot(figsize=(12, 8))
        else:
            # If we are not plotting, then only include the month of August (15 - 30 if the submission)
            # We will keep the first 15 days, just in case
            dfTS = dfTS[:]["2017-08-01":]

        return dfTS

    else:
        # print('No Data: Store: ' + str(storeNumber) + ' - ' + str(itemNumber) + '\n')
        return dfTS

In [18]:
dfTopMSE.reset_index(inplace=True)

In [19]:
# Check length
len(dfTopMSE)

162722

In [20]:
# Loop through each minimum
from ast import literal_eval

# 162,722
maxLoop = len(dfTopMSE)-1
# test a few
#maxLoop = 100

optionalPlotFunctionTS = 0
exportResultsSubmissionFileName ='D:\\project\\data\\kg_corpgroc\\export\\mse_run.csv'

firstRunHeaader = True
start_time = time.time()
    
print('Total Loops: ' + str(maxLoop))
for i in range(maxLoop):
    if i%10000 == 0:
        print_elapsed_time('Index : ' + str(i) + ' - Time Elapsed', time.time() - start_time)
    
    #print('store : ' + str(i))
    #print(dfTopMSE.loc[i])
    args = fn_concat_args(dfTopMSE.loc[i]['store_nbr'],
                          dfTopMSE.loc[i]['item_nbr'],
                          literal_eval(dfTopMSE.loc[i]['model_order']),
                          literal_eval(dfTopMSE.loc[i]['model_seasonal_order']),
                         300)
        
    df_EstimatesRun = fn_config_timeseries(args[0])
    
    # Grab dates 2017-08-15 - 2017-08-31 and write to the submission results file.
    df_EstimatesRun.reset_index(inplace=True)
    # rename column
    df_EstimatesRun = df_EstimatesRun.rename(columns={'index': 'date'})
    df_EstimatesRun = df_EstimatesRun[['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'forecast', 'forecast_rnd']]

    # Round for issues / Data Format
    df_EstimatesRun['id'] = df_EstimatesRun['id'].astype(int)
    df_EstimatesRun['store_nbr'] = df_EstimatesRun['store_nbr'].astype(int)
    df_EstimatesRun['item_nbr'] = df_EstimatesRun['item_nbr'].astype(int)
    df_EstimatesRun['unit_sales'] = df_EstimatesRun['unit_sales'].round(4)

    with open(exportResultsSubmissionFileName, 'a') as f:
        df_EstimatesRun.to_csv(f, header=firstRunHeaader, index=False, quotechar='"')
        f.close()
        
    firstRunHeaader = False

Total Loops: 162722
Index : 0 - Time Elapsed : 0 seconds
Index : 10000 - Time Elapsed : 69 min and 34 seconds
Index : 20000 - Time Elapsed : 137 min and 68 seconds
Index : 30000 - Time Elapsed : 205 min and 103 seconds
Index : 40000 - Time Elapsed : 274 min and 137 seconds
Index : 50000 - Time Elapsed : 343 min and 172 seconds
Index : 60000 - Time Elapsed : 414 min and 207 seconds
Index : 70000 - Time Elapsed : 483 min and 241 seconds
Index : 80000 - Time Elapsed : 549 min and 275 seconds
Index : 90000 - Time Elapsed : 618 min and 309 seconds
Index : 100000 - Time Elapsed : 685 min and 343 seconds
Index : 110000 - Time Elapsed : 754 min and 377 seconds
Index : 120000 - Time Elapsed : 823 min and 411 seconds
Index : 130000 - Time Elapsed : 894 min and 447 seconds
Index : 140000 - Time Elapsed : 965 min and 482 seconds
Index : 150000 - Time Elapsed : 1036 min and 518 seconds
Index : 160000 - Time Elapsed : 1105 min and 553 seconds


TypeError: invalid type comparison